# Decision Analysis

This notebook follows `analysis_execution_blueprint.md` step-by-step.

Net Revenue (Monthly) is defined as the sum of account-month MRR (from `mrr_amount`).


## Phase 1 - Establish Revenue Baseline

Goal: confirm a structural net revenue growth slowdown exists before testing drivers.


In [ ]:
from pathlib import Path\nimport pandas as pd\n\nDATA = Path('data')\nRAW = DATA / 'raw' / 'ravenstack'\nOUT = DATA / 'processed'\nOUT.mkdir(parents=True, exist_ok=True)\n

In [ ]:
accounts = pd.read_csv(RAW / 'ravenstack_accounts.csv', parse_dates=['signup_date'])\nsubs = pd.read_csv(RAW / 'ravenstack_subscriptions.csv', parse_dates=['start_date','end_date'])\nchurn = pd.read_csv(RAW / 'ravenstack_churn_events.csv', parse_dates=['churn_date'])\naccounts.shape, subs.shape, churn.shape\n

In [ ]:
def month_floor(s):\n    return s.dt.to_period('M').dt.to_timestamp()\n\ns = subs.dropna(subset=['account_id','start_date','end_date','mrr_amount']).copy()\ns['start_month'] = month_floor(s['start_date'])\ns['end_month'] = month_floor(s['end_date'])\nrows = []\nfor r in s.itertuples(index=False):\n    m = r.start_month\n    while m <= r.end_month:\n        rows.append({'account_id': r.account_id, 'month': m, 'mrr_amount': float(r.mrr_amount)})\n        m = (pd.Timestamp(m) + pd.offsets.MonthBegin(1)).normalize()\naccount_month = pd.DataFrame(rows)\naccount_month = account_month.groupby(['account_id','month'], as_index=False).agg(mrr_amount=('mrr_amount','sum')).sort_values(['month','account_id'])\naccount_month.to_csv(OUT / 'account_month_mrr.csv', index=False)\naccount_month.head()\n

In [ ]:
monthly = account_month.groupby('month', as_index=False).agg(net_revenue=('mrr_amount','sum'), active_accounts=('account_id','nunique'), arpa=('mrr_amount','mean')).sort_values('month')\nmonthly['mom_growth'] = monthly['net_revenue'].pct_change()\nmonthly['yoy_growth'] = monthly['net_revenue'].pct_change(12)\nmonthly.to_csv(OUT / 'monthly_net_revenue.csv', index=False)\nmonthly.tail()\n

### Phase 1 interpretation (minimum narrative)

- Confirm whether growth deceleration is a **rate** slowdown (YoY/MoM) versus a revenue decline.
- If YoY decelerates while MoM remains volatile, base effects may explain part of the slowdown; the purpose of later phases is to identify the dominant *pressure* in the most recent window.


## Phase 2 - Independent Hypothesis Evaluation

### Hypothesis A - Acquisition Output Decline

CAC and conversion are not available. This hypothesis is evaluated via new account volume, referral source mix, and starting MRR for new accounts.


In [ ]:
new_accounts = accounts.assign(signup_month=accounts['signup_date'].dt.to_period('M').dt.to_timestamp()) \
        .groupby('signup_month', as_index=False).agg(new_accounts=('account_id','nunique')) \
        .rename(columns={'signup_month':'month'}).sort_values('month')
new_accounts.tail()


In [ ]:
mix = accounts.assign(signup_month=accounts['signup_date'].dt.to_period('M').dt.to_timestamp()) \
        .groupby(['signup_month','referral_source'], as_index=False).agg(new_accounts=('account_id','nunique')) \
        .rename(columns={'signup_month':'month'}).sort_values(['month','new_accounts'], ascending=[True, False])
mix.head()


In [ ]:
first_mrr = account_month.sort_values(['account_id','month']).groupby('account_id', as_index=False).first()[['account_id','month','mrr_amount']] \
        .rename(columns={'month':'first_mrr_month','mrr_amount':'starting_mrr'})
starting = accounts.assign(signup_month=accounts['signup_date'].dt.to_period('M').dt.to_timestamp()) \
        .merge(first_mrr, on='account_id', how='left') \
        .groupby('signup_month', as_index=False).agg(avg_starting_mrr=('starting_mrr','mean'), median_starting_mrr=('starting_mrr','median')) \
        .rename(columns={'signup_month':'month'}).sort_values('month')
starting.tail()


#### Hypothesis A interpretation (minimum narrative)

- Use new-account volume + starting MRR + referral mix to test whether acquisition output/value are deteriorating structurally.


### Hypothesis B - Customer Lifetime Value Deterioration

This hypothesis is evaluated via churn, inferred expansion and contraction from account-month MRR deltas, and an NRR-style signal.
Churn impact timing uses prior-month MRR (t-1) for accounts that churn in month t.


In [ ]:
bridge = pd.read_csv(OUT / 'hypB_revenue_bridge_components.csv', parse_dates=['month'])
bridge.tail()


In [ ]:
churn_overall = pd.read_csv(OUT / 'hypB_churn_rate_overall.csv', parse_dates=['month'])
churn_overall.tail()


In [ ]:
churn_tenure = pd.read_csv(OUT / 'hypB_churn_by_tenure_bucket.csv', parse_dates=['month'])
churn_tenure.head()


#### Hypothesis B interpretation (minimum narrative)

- Use churn + contraction vs expansion (revenue bridge) to quantify whether post-acquisition value is decaying.
- This feeds the **pressure** track in Phase 3 (retention drag).


### Hypothesis C - Pricing Compression (Proxies)

Discount percentage is not available. Pricing compression is evaluated via ARPA drift, plan tier mix shifts, and seat migration.


In [ ]:
arpa = pd.read_csv(OUT / 'hypC_arpa_drift.csv', parse_dates=['month'])
arpa.tail()


In [ ]:
tier_mix = pd.read_csv(OUT / 'hypC_plan_tier_mix.csv', parse_dates=['month'])
tier_mix.head()


In [ ]:
seats = pd.read_csv(OUT / 'hypC_seat_migration.csv', parse_dates=['month'])
seats.tail()


#### Hypothesis C interpretation (minimum narrative)

- Use ARPA drift + plan tier mix + seats as pricing/packaging proxies (discounts are not observed in this dataset).
- In Phase 3, pricing *pressure* is only counted when ARPA drift is negative (compression/headwind).


## Phase 3 - Structural Comparison

Compare drivers using two complementary signals:
- **Lever** (magnitude): largest absolute movement over the rolling window
- **Pressure** (directional drag): headwind-only contributions over the rolling window

Dominance is assessed on the **pressure** track using the explicit consistency rule.


In [ ]:
comp = pd.read_csv(OUT / 'phase3_driver_comparison.csv', parse_dates=['month'])
comp.tail()


## Figures

Key figures are saved under `docs/figures/`.
- Net revenue trend: `docs/figures/net_revenue_trend.png`
- Revenue bridge components: `docs/figures/revenue_bridge_components.png`
- Plan tier mix: `docs/figures/plan_tier_mix_account_share.png`


### Phase 3 interpretation (minimum narrative)

Use **pressure** to identify what is actively dragging net revenue (headwind-only).
Use **lever** to identify what is moving the most (largest magnitude), regardless of whether it is helping or hurting.

If pressure leadership does not hold for >=3 consecutive months (or fails the material margin gate), treat the window as **mixed-signal** and avoid overconfident single-driver recommendations.

As of **Dec 2024**, rolling 3-month pressure is highest for **retention**: retention=82,016, acquisition=32,688, pricing=857.
This supports treating retention as the dominant near-term headwind even when the formal >=3-month dominance rule yields a mixed-signal conclusion.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

comp = pd.read_csv(OUT / 'phase3_driver_comparison.csv', parse_dates=['month'])

plt.figure(figsize=(10,4))
plt.plot(comp['month'], comp.get('acq_pressure_3m'), label='Acquisition pressure (3m)')
plt.plot(comp['month'], comp.get('ret_pressure_3m'), label='Retention pressure (3m)')
plt.plot(comp['month'], comp.get('prc_pressure_3m'), label='Pricing pressure (3m)')
plt.title('Directional pressure (rolling 3-month headwind)')
plt.xlabel('Month')
plt.ylabel('MRR impact (proxy)')
plt.legend()
plt.tight_layout()
plt.show()
